# Can Bike CO Data Cleaning

This notebook performs the data treatments we outline in the paper, narrowing the dataset down from it's original size to the 122 users and 61,496 trips we analyze in the paper. The csv from TSDC can be inputted to the notebook as "trip_program.csv" and will be transformed and saved as "filtered_and_merged_trips.csv". This filtered and merged file is what can be used as input to the analysis notebook. 

## Setup: dependencies and reading in raw file

In [47]:
#dependencies

import pandas as pd
import numpy as np

from collections import defaultdict

In [48]:
# Loading mapping dictionaries from mapping_dictionaries notebook
%store -r df_ei
%store -r dic_re
%store -r dic_pur
%store -r dic_fuel

# convert a dictionary to a defaultdict
dic_re = defaultdict(lambda: 'Other',dic_re)
dic_pur = defaultdict(lambda: 'Other',dic_pur)
dic_fuel = defaultdict(lambda: 'Other',dic_fuel)

In [49]:
#read in the raw csv -- all labeled trips?
data = pd.read_csv("trip_program.csv")
print(len(data)) #92395
data['user_id'].nunique() #235

92395


235

## Filter out stage & minipilot

In [50]:
#count the stage and mini users
stage_data = data[data.program=='stage']
print(stage_data.user_id.nunique())
print(len(stage_data))

mini_data = data[data.program=='prepilot']
print(mini_data.user_id.nunique())
print(len(mini_data))

59
10827
12
2374


In [51]:
#filter out stage and mini pilot users and trips from the full trip dataset
data_non_stage = data[data.program != "stage"]
print(len(data_non_stage))
data_non_stage_non_mini = data_non_stage[data_non_stage.program != "prepilot"]
print(len(data_non_stage_non_mini))

81568
79194


In [52]:
data_non_stage_non_mini['user_id'].nunique()

164

In [53]:
data_non_stage_non_mini['program'].unique()

array(['4c', 'cc', 'fc', 'pc', 'sc', 'vail'], dtype=object)

## Merge with the Sociodemographic Data

In [54]:
expanded_ct = data.copy()
socio_data = pd.read_csv("Can Do Colorado eBike Program - en.csv")
socio_data.rename(columns={'Unique User ID (auto-filled, do not edit)':'user_id',
                          'Please identify which category represents your total household income, before taxes, for last year.':'HHINC',
                          'How many motor vehicles are owned, leased, or available for regular use by the people who currently live in your household?':'VEH',
                           'In which year were you born?':'AGE',
                          'Including yourself, how many people live in your home?':'HHSIZE',
                          'How many children under age 18 live in your home?':'CHILDREN',
                          'What is your gender?':'GENDER',
                          'If you were unable to use your household vehicle(s), which of the following options would be available to you to get you from place to place?':'available_modes',
                          'Are you a student?':'STUDENT'}, inplace=True)
socio_data = socio_data[~socio_data.user_id.isnull()]

In [55]:
# socio_data.head() #use to check the data

In [56]:
# Deal with people who have multiple responses by using most recent
socio_data = socio_data.sort_values(by=['user_id', 'Timestamp'])
socio_data.drop_duplicates(subset=['user_id'], keep='last', inplace=True)
socio_data['user_id_socio'] = socio_data.user_id
socio_data = socio_data.drop(labels='user_id', axis=1)

# socio_data.head()

In [57]:
# Lose some trips due to people with no survey responses
expanded_ct['user_id_socio'] = expanded_ct.user_id.astype(str)
expanded_ct.user_id_socio = [i.replace('-','') for i in expanded_ct.user_id_socio] # remove all dashes from strings
expanded_ct = expanded_ct.merge(socio_data, on='user_id_socio')
print(len(expanded_ct))
print(expanded_ct.user_id_socio.nunique())
expanded_ct.program.unique()


# expanded_ct.head()
#optionally save the csv at this point
#expanded_ct.to_csv("expanded_ct.csv")

81547
170


array(['stage', '4c', 'cc', 'fc', 'pc', 'sc', 'vail', 'prepilot'],
      dtype=object)

## Filtering the whole dataset

In [58]:
data = expanded_ct.copy()

#separating programs
four_corners = data[data.program == "4c"]
community_cycles = data[data.program == "cc"]
fort_collins = data[data.program == "fc"]
pueblo = data[data.program == "pc"]
smart_commute = data[data.program == "sc"]
vail = data[data.program == "vail"]

print(four_corners['user_id'].nunique())
print(community_cycles['user_id'].nunique())
print(fort_collins['user_id'].nunique())
print(pueblo['user_id'].nunique())
print(smart_commute['user_id'].nunique())
print(vail['user_id'].nunique())

print(len(four_corners))
print(len(community_cycles))
print(len(fort_collins))
print(len(pueblo))
print(len(smart_commute))
print(len(vail))

13
46
29
36
14
9
4456
28056
11751
17767
8431
5223


In [15]:
# Optionally initialize from saved dataset
#data = pd.read_csv("expanded_ct.csv")
# Or continue from data loaded from the database
data = expanded_ct.copy()

# Get timestamp from known year/month/day aggregated to days
data.rename(columns={'start_local_dt_year':'year','start_local_dt_month':'month','start_local_dt_day':'day'}, inplace=True)
data['date_time'] = pd.to_datetime(data[['year','month','day']])

# Fix age (birth year to age)
data['AGE'] = 2022 - data['AGE']

# Number of workers (size of HH - kids)
data['WORKERS'] = data['HHSIZE'] - data['CHILDREN']

# Duration in minutes (hours to minutes)
data['duration'] = data['duration'] / 60

# E-bike/not E-Bike variable
data['is_ebike'] = "E-Bike Trips"
data.loc[data['Mode_confirm']!="E-bike", 'is_ebike'] = "Non E-Bike Trips"

data = data[~data['HHINC'].isin(['Prefer not to say', '$150,000'])] # Side note why is 150k (n=7) its own bin?
data['HHINC_NUM'] = data.HHINC.replace(['Less than $24,999',
                                       '$25,000-$49,999',
                                       '$50,000-$99,999',
                                       '$100,000 -$149,999',
                                       '$150,000-$199,999',
                                       '$200,000 or more'], [12500,37500,75000,125000,175000,250000])

# Calculate average income per adult in the household
data['PINC'] = data['HHINC_NUM'] / data['WORKERS']

# Combine variable categories
data = data.replace('Gas Car, drove alone', 'Car')
data = data.replace('Gas Car, with others', 'Shared Car')
data = data.replace('Bikeshare', 'Shared Micromobility')
data = data.replace('Scooter share', 'Shared Micromobility')
data = data.replace('Regular Bike', 'Personal Micromobility')
data = data.replace('Skate board', 'Personal Micromobility')
data = data.replace('Train', 'Transit')
data = data.replace('Free Shuttle', 'Transit')
data = data.replace('Bus', 'Transit')
data = data.replace('Walk', 'Walk')
data = data.replace('Taxi/Uber/Lyft', 'Ridehail')
data = data.replace('Pilot ebike', 'E-Bike')

# Categorical type will include all days/modes in groupby even if there is no data for a particular tabulation
data.user_id = pd.Categorical(data.user_id)
data.date_time = pd.Categorical(data.date_time)
data.mode_confirm = pd.Categorical(data.mode_confirm, ordered=True, categories=np.unique(list(dic_re.keys())))

# Add order to categorical variables
data.HHINC = pd.Categorical(data.HHINC, ordered=True, categories=['Less than $24,999',
                                                                 '$25,000-$49,999',
                                                                 '$50,000-$99,999'])
data['Mode'] = pd.Categorical(data.Mode_confirm, ordered=True, categories=[
    'E-bike',
    'Car',
    'Shared Car',
    'Walk',
    'Transit',
    'Personal Micromobility',
    'Shared Micromobility',
    'Ridehail',
    'Other'])
data.VEH = pd.Categorical(data.VEH, ordered=True, categories=['0','1','2','3','4+'])
data['PINC_NUM'] = data['PINC']
data.PINC = pd.cut(data.PINC, bins=[0,10000,20000,30000,40000,50000,60000,70000,999999],
                  labels=["$0-9",
                         "$10-19",
                         "$20-29",
                         "$30-39",
                         "$40-49",
                         "$50-59",
                         "$60-69",
                         "$70+"])

# Vehicles per driver
data['VEH_num'] = data['VEH'].replace(['1','2','3','4+'],[1,2,3,4]).astype(int)
data['DRIVERS'] = data["Including yourself, how many people have a driver's license in your household?"]
data['DRIVERS_num'] = data['DRIVERS'].replace
data['veh_per_driver'] = (data['VEH_num'] / data['DRIVERS']).fillna(0)
data.loc[data['veh_per_driver']==np.inf, 'veh_per_driver'] = 0

print(len(data))

74017


In [16]:
#filtered out ages that were greater than 100
data = data[data['AGE'] < 100]
#filter out durations longer than 8 hours
data = data[data['duration']<480]
#distances more than 50 miles 
data = data[data['distance_miles']<50]
#records that had ’prefer not to say’ as a response for household income, household vehicles, and other available modes
data = data[~data['HHINC'].isin(['Prefer not to say','$100,000 -$149,999','$150,000','$150,000-$199,999','$200,000 or more'])] # Side note why is 150k (n=7) its own bin?
data = data[~data['VEH'].isin(['Prefer not to say / Prefiero no decir.'])]
data = data[~data['available_modes'].isin(['None', 'Prefer not to say'])]

#filter household sizes smaller than the number of kids
data = data[data['HHSIZE']>data['CHILDREN']]
#filter out households greater than 10
data = data[data['HHSIZE']<10]

print(len(data))

69848


In [17]:
#filter out 'not a trip' trips
data = data[~data['Mode_confirm'].isin(['Not a Trip'])]
data = data[~data['Replaced_mode'].isin(['Not a Trip'])]
data = data[~data['Trip_purpose'].isin(['not_a_trip'])]

print(len(data))

a = data[data['AGE']>100]
print(len(a)) #should be 0

67686
0


In [18]:
## checking minipilot post-filtering numbers
mini_data = data[data.program == 'prepilot']
print(len(mini_data))
print(mini_data.user_id.nunique())


194
1


## Filter out trips prior to user's 1st Ebike Trip - done by program


In [19]:
#separating programs
four_corners = data[data.program == "4c"]
community_cycles = data[data.program == "cc"]
fort_collins = data[data.program == "fc"]
pueblo = data[data.program == "pc"]
smart_commute = data[data.program == "sc"]
vail = data[data.program == "vail"]

print(four_corners['user_id'].nunique())
print(community_cycles['user_id'].nunique())
print(fort_collins['user_id'].nunique())
print(pueblo['user_id'].nunique())
print(smart_commute['user_id'].nunique())
print(vail['user_id'].nunique())

print(len(four_corners))
print(len(community_cycles))
print(len(fort_collins))
print(len(pueblo))
print(len(smart_commute))
print(len(vail))

10
43
26
32
13
8
3356
25409
10952
13072
6886
4321


In [20]:
#filtering each of them
from datetime import datetime

In [21]:
#smart commute filtering

#timestamp conversion
smart_commute['start_ts']= pd.to_datetime(smart_commute['start_ts'], utc=True, unit='s')

#grouping, counting unique users
trip_sep=smart_commute.groupby(['user_id','Mode_confirm']).apply(lambda x:x[x.start_ts==min(x.start_ts)])
print(trip_sep['user_id'].nunique())

#consider only trips with E-bike (to get first e-bike trip)
sc_ebike_first=trip_sep[trip_sep['Mode_confirm']=='E-bike']

#get all the trips by ysers who ever had an e-bike trip
sc_ebike_user_list= sc_ebike_first['user_id'].tolist()
smart_commute_incl_ebike = smart_commute[smart_commute['user_id'].isin(sc_ebike_user_list)]
print(smart_commute_incl_ebike['user_id'].nunique())

#filter to the earliest ebike trip
for unique_id in sc_ebike_first['user_id']:
    for date in sc_ebike_first['start_ts']:
        smart_commute_ebike_first=smart_commute_incl_ebike[(smart_commute_incl_ebike['start_ts'] >= date)]

sc_unique_ebikefirst=smart_commute_ebike_first['user_id'].unique()
print(smart_commute_ebike_first['user_id'].nunique()) #11

/tmp/ipykernel_171/3643144510.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smart_commute['start_ts']= pd.to_datetime(smart_commute['start_ts'], utc=True, unit='s')


13
11
11


In [22]:
#filter four corners
four_corners['start_ts']= pd.to_datetime(four_corners['start_ts'], utc=True, unit='s')

trip_sep_fc=four_corners.groupby(['user_id','Mode_confirm']).apply(lambda x:x[x.start_ts==min(x.start_ts)])
print(trip_sep_fc['user_id'].nunique())

fc_ebike_first=trip_sep_fc[trip_sep_fc['Mode_confirm']=='E-bike']

fc_ebike_user_list= fc_ebike_first['user_id'].tolist()
four_corners_incl_ebike = four_corners[four_corners['user_id'].isin(fc_ebike_user_list)]
print(four_corners_incl_ebike['user_id'].nunique())

for unique_id in fc_ebike_first['user_id']:
    for date in fc_ebike_first['start_ts']:
        four_corners_ebike_first=four_corners_incl_ebike[(four_corners_incl_ebike['start_ts'] >= date)]
        
fc_unique_ebikefirst=four_corners_ebike_first['user_id'].unique()
print(four_corners_ebike_first['user_id'].nunique())

10


/tmp/ipykernel_171/4259660508.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  four_corners['start_ts']= pd.to_datetime(four_corners['start_ts'], utc=True, unit='s')


10
10


In [23]:
#filtering community cycles
community_cycles['start_ts']= pd.to_datetime(community_cycles['start_ts'], utc=True, unit='s')

trip_sep_cc=community_cycles.groupby(['user_id','Mode_confirm']).apply(lambda x:x[x.start_ts==min(x.start_ts)])
print(trip_sep_cc['user_id'].nunique())

cc_ebike_first=trip_sep_cc[trip_sep_cc['Mode_confirm']=='E-bike']

cc_ebike_user_list= cc_ebike_first['user_id'].tolist()
community_cycles_incl_ebike = community_cycles[community_cycles['user_id'].isin(cc_ebike_user_list)]
print(community_cycles_incl_ebike['user_id'].nunique())

for unique_id in cc_ebike_first['user_id']:
    for date in cc_ebike_first['start_ts']:
        community_cycles_ebike_first=community_cycles_incl_ebike[(community_cycles_incl_ebike['start_ts'] >= date)]

cc_unique_ebikefirst=community_cycles_ebike_first['user_id'].unique()
print(community_cycles_ebike_first['user_id'].nunique())

/tmp/ipykernel_171/1138286727.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  community_cycles['start_ts']= pd.to_datetime(community_cycles['start_ts'], utc=True, unit='s')


43
42
42


In [24]:
#filtering fort collins
fort_collins['start_ts']= pd.to_datetime(fort_collins['start_ts'], utc=True, unit='s')

trip_sep_fc=fort_collins.groupby(['user_id','Mode_confirm']).apply(lambda x:x[x.start_ts==min(x.start_ts)])
print(trip_sep_fc['user_id'].nunique())

fc_ebike_first=trip_sep_fc[trip_sep_fc['Mode_confirm']=='E-bike']

fc_ebike_user_list= fc_ebike_first['user_id'].tolist()
fort_collins_incl_ebike = fort_collins[fort_collins['user_id'].isin(fc_ebike_user_list)]
print(fort_collins_incl_ebike['user_id'].nunique())

for unique_id in fc_ebike_first['user_id']:
    for date in fc_ebike_first['start_ts']:
        fort_collins_ebike_first=fort_collins_incl_ebike[(fort_collins_incl_ebike['start_ts'] >= date)]
        
fc_unique_ebikefirst=fort_collins_ebike_first['user_id'].unique()
print(fort_collins_ebike_first['user_id'].nunique())

/tmp/ipykernel_171/1774079327.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fort_collins['start_ts']= pd.to_datetime(fort_collins['start_ts'], utc=True, unit='s')


26
22
22


In [25]:
#filtering pueblo
pueblo['start_ts']= pd.to_datetime(pueblo['start_ts'], utc=True, unit='s')

trip_sep_pu=pueblo.groupby(['user_id','Mode_confirm']).apply(lambda x:x[x.start_ts==min(x.start_ts)])
print(trip_sep_pu['user_id'].nunique())

pu_ebike_first=trip_sep_pu[trip_sep_pu['Mode_confirm']=='E-bike']

pu_ebike_user_list= pu_ebike_first['user_id'].tolist()
pueblo_incl_ebike = pueblo[pueblo['user_id'].isin(pu_ebike_user_list)]
print(pueblo_incl_ebike['user_id'].nunique())

for unique_id in pu_ebike_first['user_id']:
    for date in pu_ebike_first['start_ts']:
        pueblo_ebike_first=pueblo_incl_ebike[(pueblo_incl_ebike['start_ts'] >= date)]
        
pu_unique_ebikefirst=pueblo_ebike_first['user_id'].unique()
print(pueblo_ebike_first['user_id'].nunique())

/tmp/ipykernel_171/950012586.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pueblo['start_ts']= pd.to_datetime(pueblo['start_ts'], utc=True, unit='s')


32
29
29


In [26]:
#filtering vail
vail['start_ts']= pd.to_datetime(vail['start_ts'], utc=True, unit='s')

trip_sep_va=vail.groupby(['user_id','Mode_confirm']).apply(lambda x:x[x.start_ts==min(x.start_ts)])
print(trip_sep_va['user_id'].nunique())

va_ebike_first=trip_sep_va[trip_sep_va['Mode_confirm']=='E-bike']

va_ebike_user_list= va_ebike_first['user_id'].tolist()
vail_incl_ebike = vail[vail['user_id'].isin(va_ebike_user_list)]
print(vail_incl_ebike['user_id'].nunique())

for unique_id in va_ebike_first['user_id']:
    for date in va_ebike_first['start_ts']:
        vail_ebike_first=vail_incl_ebike[(vail_incl_ebike['start_ts'] >= date)]
        
va_unique_ebikefirst=vail_ebike_first['user_id'].unique()
print(vail_ebike_first['user_id'].nunique())

8
8


/tmp/ipykernel_171/3916199498.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vail['start_ts']= pd.to_datetime(vail['start_ts'], utc=True, unit='s')


8


In [27]:
## Recombine the Filtered Programs

In [28]:
#checking num users and num trips in each program
print(four_corners_ebike_first['user_id'].nunique())
print(community_cycles_ebike_first['user_id'].nunique())
print(fort_collins_ebike_first['user_id'].nunique())
print(pueblo_ebike_first['user_id'].nunique())
print(smart_commute_ebike_first['user_id'].nunique())
print(vail_ebike_first['user_id'].nunique())

print(len(four_corners_ebike_first))
print(len(community_cycles_ebike_first))
print(len(fort_collins_ebike_first))
print(len(pueblo_ebike_first))
print(len(smart_commute_ebike_first))
print(len(vail_ebike_first))

10
42
22
29
11
8
2240
25151
10656
12620
6515
4314


In [29]:
#combining the filtered datasets
filtered_merged = pd.concat([four_corners_ebike_first, community_cycles_ebike_first, fort_collins_ebike_first, 
                             pueblo_ebike_first, smart_commute_ebike_first,vail_ebike_first], axis=0)
print(len(filtered_merged))
print(filtered_merged['user_id'].nunique())

61496
122


## Save to CSV for use in Analysis Notebooks

In [25]:
filtered_merged.columns

Index(['source', 'end_ts', 'end_fmt_time', 'end_loc', 'raw_trip', 'start_ts',
       'start_fmt_time', 'start_loc', 'duration', 'distance', 'start_place',
       'end_place', 'cleaned_trip', 'inferred_labels', 'inferred_trip',
       'expectation', 'confidence_threshold', 'expected_trip', 'user_input',
       'year', 'month', 'day', 'start_local_dt_hour', 'start_local_dt_minute',
       'start_local_dt_second', 'start_local_dt_weekday',
       'start_local_dt_timezone', 'end_local_dt_year', 'end_local_dt_month',
       'end_local_dt_day', 'end_local_dt_hour', 'end_local_dt_minute',
       'end_local_dt_second', 'end_local_dt_weekday', 'end_local_dt_timezone',
       '_id', 'user_id', 'metadata_write_ts', 'mode_confirm',
       'purpose_confirm', 'replaced_mode', 'distance_miles', 'Mode_confirm',
       'Replaced_mode', 'Trip_purpose', 'user_id_str', 'program', 'opcode',
       'distance_km', 'user_id_socio', 'Timestamp', 'AGE', 'GENDER',
       'Do you have a valid driver's license?', 

In [26]:
#save as a csv, to be used as input to analysis!
filtered_merged.to_csv("filtered_merged_trips.csv")

122


,distance_miles,duration
count,61496.000000,61496.000000
mean,3.933690,24.145146
std,5.991457,30.691071
min,0.062013,0.000068
25%,0.721590,9.173613
50%,1.983511,15.289998
75%,4.342240,28.071609
max,49.929320,479.495935
